In [1]:
import numpy as np
import os
import time
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model,Sequential
from keras.utils import np_utils
from sklearn import preprocessing
from keras.layers import *
from keras.models import *
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras.layers import Convolution2D, GlobalAveragePooling2D,Dropout
from keras.layers.recurrent import LSTM
import glob
from keras import optimizers
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
X_data1_test = []
image = np.empty(48, dtype=object)
files = glob.glob("/content/drive/My Drive/ALL/ALL_IDB1/im_0_test/*.jpg")
for myFile in files:
  image = cv2.imread(myFile)
  x_img1_test = cv2.resize(image,(224,224),interpolation = cv2.INTER_CUBIC)
  X_data1_test.append(x_img1_test)
print(np.array(X_data1_test).shape)  


(17, 224, 224, 3)


In [4]:
X_data2_test = []
image = np.empty(48, dtype=object)
files = glob.glob("/content/drive/My Drive/ALL/ALL_IDB1/im_1_test/*.jpg")
for myFile in files:
  image = cv2.imread(myFile)
  x_img2_test = cv2.resize(image,(224,224),interpolation = cv2.INTER_CUBIC)
  X_data2_test.append(x_img2_test)
print(np.array(X_data2_test).shape)  


(14, 224, 224, 3)


In [5]:
X_data1_train = []
image = np.empty(48, dtype=object)
files = glob.glob("/content/drive/My Drive/ALL/ALL_IDB1/im_0_train/*.jpg")
for myFile in files:
  image = cv2.imread(myFile)
  x_img1_train = cv2.resize(image,(224,224),interpolation = cv2.INTER_CUBIC)
  X_data1_train.append(x_img1_train)
print(np.array(X_data1_train).shape)  


(42, 224, 224, 3)


In [6]:
X_data2_train = []
image = np.empty(48, dtype=object)
files = glob.glob("/content/drive/My Drive/ALL/ALL_IDB1/im_1_train/*.jpg")
for myFile in files:
  image = cv2.imread(myFile)
  x_img2_train = cv2.resize(image,(224,224),interpolation = cv2.INTER_CUBIC)
  X_data2_train.append(x_img2_train)
print(np.array(X_data2_train).shape)  


(35, 224, 224, 3)


In [0]:
for i in X_data2_test:
  X_data1_test.append(i)
for j in X_data2_train:
  X_data1_train.append(j)

In [8]:
print(np.array(X_data1_train).shape) 
print(np.array(X_data1_test).shape) 


(77, 224, 224, 3)
(31, 224, 224, 3)


In [0]:
labels1 = np.ones(len(X_data1_train),dtype='int64')
labels1[0:41] = 0
labels1[42:] = 1
labels2 = np.ones(len(X_data1_test),dtype='int64')
labels2[0:16] = 0
labels2[17:] = 1

In [0]:
X_train = X_data1_train
X_train = np.array(X_train)
X_test = X_data1_test
X_test = np.array(X_test)
Y_train = np_utils.to_categorical(labels1, 2)
Y_train = np.array(Y_train)
Y_test = np_utils.to_categorical(labels2, 2)
Y_test = np.array(Y_test)

In [11]:
base_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))
base_model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [12]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(2, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [0]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.7, patience=6, min_lr=0.000001, verbose=1),
    ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/model-ResNet50.h5', verbose=1, save_best_only=True)
]

In [15]:
t=time.time()
#	t = now()
hist = model.fit(X_train, Y_train, batch_size=16, epochs=13, verbose=1,callbacks = callbacks, validation_data=(X_test, Y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=4, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 77 samples, validate on 31 samples
Epoch 1/13
77/77 [==============================] - 98s 1s/step - loss: 0.5600 - acc: 0.7403 - val_loss: 0.8520 - val_acc: 0.3871

Epoch 00001: val_loss improved from inf to 0.85202, saving model to /content/drive/My Drive/Colab Notebooks/model-ResNet50.h5
Epoch 2/13
77/77 [==============================] - 90s 1s/step - loss: 0.2824 - acc: 0.9481 - val_loss: 0.7024 - val_acc: 0.7097

Epoch 00002: val_loss improved from 0.85202 to 0.70241, saving model to /content/drive/My Drive/Colab Notebooks/model-ResNet50.h5
Epoch 3/13
77/77 [==============================] - 90s 1s/step - loss: 0.1885 - acc: 0.9610 - val_loss: 0.4183 - val_acc: 0.8710

Epoch 00003: val_loss improved from 0.70241 to 0.41834, saving model to /content/drive/My Drive/Colab Notebooks/model-ResNet50.h5
Epoch 4/13
77/77 [==============================] - 89s 1s/step - loss: 0.1422 - ac